In [ ]:
import numpy as np
import random
import collections
import time
import matplotlib.pyplot as plt

n_values = [int(input("Enter the value of n (size of the puzzle grid): "))]
def generate_puzzle(n):
    numbers = list(range(n * n))
    random.shuffle(numbers)
    puzzle = np.array(numbers).reshape(n, n)
    return puzzle

def goal_test(puzzle):
    n = puzzle.shape[0]
    goal = np.arange(1, n * n)
    goal = np.append(goal, 0)
    return np.array_equal(puzzle.flatten(), goal)

def find_zero(state):
    return tuple(np.argwhere(state == 0)[0])

def move_tile(puzzle, direction):
    row, col = find_zero(puzzle)
    new_puzzle = puzzle.copy()
    if direction == "up" and row > 0:
        new_puzzle[row, col], new_puzzle[row - 1, col] = new_puzzle[row - 1, col], new_puzzle[row, col]
    elif direction == "down" and row < n - 1:
        new_puzzle[row, col], new_puzzle[row + 1, col] = new_puzzle[row + 1, col], new_puzzle[row, col]
    elif direction == "left" and col > 0:
        new_puzzle[row, col], new_puzzle[row, col - 1] = new_puzzle[row, col - 1], new_puzzle[row, col]
    elif direction == "right" and col < n - 1:
        new_puzzle[row, col], new_puzzle[row, col + 1] = new_puzzle[row, col + 1], new_puzzle[row, col]
    else:
        return None
    return new_puzzle

def bfs_solve(puzzle):
    queue = collections.deque([puzzle])
    visited = set()
    while queue:
        current_state = queue.popleft()
        state_tuple = tuple(current_state.flatten())
        if state_tuple in visited:
            continue
        visited.add(state_tuple)
        if goal_test(current_state):
            return True
        for move in ["left", "right", "up", "down"]:
            new_state = move_tile(current_state, move)
            if new_state is not None and tuple(new_state.flatten()) not in visited:
                queue.append(new_state)
    return False

def dfs_solve(puzzle):
    stack = [puzzle]
    visited = set()
    while stack:
        current_state = stack.pop()
        state_tuple = tuple(current_state.flatten())
        if state_tuple in visited:
            continue
        visited.add(state_tuple)
        if goal_test(current_state):
            return True
        for move in ["left", "right", "up", "down"]:
            new_state = move_tile(current_state, move)
            if new_state is not None and tuple(new_state.flatten()) not in visited:
                stack.append(new_state)
    return False

def id_solve(puzzle):
    depth = 0
    while True:
        stack = [(puzzle, 0)]
        visited = set()
        while stack:
            current_state, current_depth = stack.pop()
            state_tuple = tuple(current_state.flatten())
            if state_tuple in visited:
                continue
            visited.add(state_tuple)
            if goal_test(current_state):
                return True
            if current_depth < depth:
                for move in ["left", "right", "up", "down"]:
                    new_state = move_tile(current_state, move)
                    if new_state is not None and tuple(new_state.flatten()) not in visited:
                        stack.append((new_state, current_depth + 1))
        depth += 1

for n in n_values:
    bfs_avg_time = 0
    dfs_avg_time = 0
    id_avg_time = 0
    valid_bfs = valid_dfs = valid_id = 0

    for _ in range(10):
        puzzle = generate_puzzle(n)
        
        start_time = time.time()
        if bfs_solve(puzzle):
            bfs_avg_time += time.time() - start_time
            valid_bfs += 1
        
        start_time = time.time()
        if dfs_solve(puzzle):
            dfs_avg_time += time.time() - start_time
            valid_dfs += 1

        start_time = time.time()
        if id_solve(puzzle):
            id_avg_time += time.time() - start_time
            valid_id += 1
    
    bfs_times.append(bfs_avg_time / valid_bfs if valid_bfs > 0 else 0)
    dfs_times.append(dfs_avg_time / valid_dfs if valid_dfs > 0 else 0)
    id_times.append(id_avg_time / valid_id if valid_id > 0 else 0)

plt.figure(figsize=(10, 6))
plt.plot(n_values, bfs_times, label="BFS")
plt.plot(n_values, dfs_times, label="DFS")
plt.plot(n_values, id_times, label="ID")
plt.xlabel("n (Puzzle Size)")
plt.ylabel("Average Time (Seconds)")
plt.legend()
plt.title("Comparison of BFS, DFS, and ID for Puzzle Solving")
plt.show()












#include <stdio.h>
#include <stdbool.h>
#include <stdlib.h>

// Function to find the minimum operations to obtain
// d liters in one jug
int minSteps(int m, int n, int d) {
  
    // Check if the target is achievable
    if (d > (m > n ? m : n)) return -1; 
    
      //  Queue for BFS: each state is (jug1, jug2, steps)
    int q[10000][3], front = 0, rear = 0;

    // For tracking the visited states, initialized to false
    bool visited[m + 1][n + 1];
    for (int i = 0; i <= m; i++) {
        for (int j = 0; j <= n; j++) {
            visited[i][j] = false;
        }
    }

    // Start with both jugs empty, pushing the initial
    // state (0, 0, 0) 
    q[rear][0] = 0;
    q[rear][1] = 0;
    q[rear][2] = 0;
    rear++;
    visited[0][0] = true;

    while (front != rear) {
      
        // Extract the front of the queue
        int jug1 = q[front][0];
        int jug2 = q[front][1];
        int steps = q[front][2];
        front++;

        if (jug1 == d || jug2 == d) return steps;

        // 1: Fill jug1 to its maximum capacity
        if (!visited[m][jug2]) {
            visited[m][jug2] = true;
            q[rear][0] = m;
            q[rear][1] = jug2;
            q[rear][2] = steps + 1;
            rear++;
        }

        // 2: Fill jug2 to its maximum capacity
        if (!visited[jug1][n]) {
            visited[jug1][n] = true;
            q[rear][0] = jug1;
            q[rear][1] = n;
            q[rear][2] = steps + 1;
            rear++;
        }

        // 3: Empty jug1
        if (!visited[0][jug2]) {
            visited[0][jug2] = true;
            q[rear][0] = 0;
            q[rear][1] = jug2;
            q[rear][2] = steps + 1;
            rear++;
        }

        // 4: Empty jug2
        if (!visited[jug1][0]) {
            visited[jug1][0] = true;
            q[rear][0] = jug1;
            q[rear][1] = 0;
            q[rear][2] = steps + 1;
            rear++;
        }

        // 5: Pour jug1 into jug2
        int pour1to2 = jug1 < (n - jug2) ? jug1 : (n - jug2);
        if (!visited[jug1 - pour1to2][jug2 + pour1to2]) {
            visited[jug1 - pour1to2][jug2 + pour1to2] = true;
            q[rear][0] = jug1 - pour1to2;
            q[rear][1] = jug2 + pour1to2;
            q[rear][2] = steps + 1;
            rear++;
        }

        // 6: Pour jug2 into jug1
        int pour2to1 = jug2 < (m - jug1) ? jug2 : (m - jug1);
        if (!visited[jug1 + pour2to1][jug2 - pour2to1]) {
            visited[jug1 + pour2to1][jug2 - pour2to1] = true;
            q[rear][0] = jug1 + pour2to1;
            q[rear][1] = jug2 - pour2to1;
            q[rear][2] = steps + 1;
            rear++;
        }
    }

    return -1;
}

int main() {
  
    // jug1 = 4 litre, jug2 = 3 litre 
    int m = 4, n = 3, d = 2;
    int result = minSteps(m, n, d);
    printf("%d\n", result);
    return 0;
}
